In [ ]:
import sys; sys.path.append("..")
import IPython
if (ipy:= IPython.get_ipython()) is not None:
    ipy.run_line_magic("load_ext", "autoreload")
    ipy.run_line_magic("autoreload", "2")
import numpy as np
from scipy.optimize import minimize, Bounds
import time, os

import ElasticRods
import elastic_rods
from elastic_rods import PeriodicRod, RodMaterial
from linkage_vis import LinkageViewer as Viewer

from py_newton_optimizer import NewtonOptimizerOptions
import compute_vibrational_modes
from sparse_matrices import SuiteSparseMatrix, TripletMatrix

from tencers import *

from Tencers.viewers import HybridViewer
from Tencers.springs import *
from Tencers.state_saver import save_state, load_state
from Tencers.rods_IO import *
from Tencers.init import *


This notebook runs an inverse design optimization algorithm to find the cables that allow to approximate a hypotrochoid trefoil tencer.

# Define target rods

In [ ]:
# Create target rod: hypotrochoid trefoil

n_divisions = 103

t = np.linspace(0,2*np.pi,n_divisions,endpoint=True)
t = np.concatenate([t, [t[1]]])  # last and first edge overlap
x = np.sin(t) + 2 * np.sin(2*t)
y = np.cos(t) - 2 * np.cos(2*t)
z = -np.sin(3*t)

rod_points = np.column_stack([x, y, z]) * 3.47 * 2

rod = PeriodicRod(rod_points, zeroRestCurvature=True)
rod_youngs_modulus = 13300
material = RodMaterial('ellipse', rod_youngs_modulus, 0.5, [0.1, 0.1])  
rod.setMaterial(material)
minimize_twist(rod)


print("Number of vertices: ", rod.numVertices())
print("Rod length: ", rod.restLength(), "cm")
print("Rod thickness: ", rod.rod.material(0).crossSectionHeight, "cm")
print("Rod Young modulus: ", rod_youngs_modulus, "MPa")

viewer1 = Viewer(rod, 400,300, wireframe=True)
viewer1.show()

In [ ]:
open_rods = []
closed_rods = [rod]
rods = [rod]
target_rods = [rod.rod]

# Cable initialization: remove compressed springs

In [ ]:
# Initialize springs
springs = []
attachment_vertices = []
init_stiffness = compute_initial_stiffness_pr(rod)
print("Initial spring stiffness: ", init_stiffness)

# Dense network of springs
for i in range(rod.numVertices()):
    for j in range(i+2,rod.numVertices()):
        if i == 0 and j == rod.numVertices() - 1:
            continue
        attachment_vertices.append(SpringAttachmentVertices(0,i,0,j))
        coordA = rod.deformedPoints()[i]
        coordB = rod.deformedPoints()[j]
        dist = np.linalg.norm(coordB-coordA)
        springs.append(Spring(coordA,coordB,init_stiffness,dist))
        if dist < 1e-6: # Make sure data is not corrupted and that no 2 points are at distance 0
            raise Exception("Error: intra-rod spring of length 0")

# Create initial tencer
tencer = Tencer([],[rod],springs,attachment_vertices, [rod.rod])

In [ ]:
# Optimizer options (equilibrium)
opt = NewtonOptimizerOptions()
opt.useNegativeCurvatureDirection = True
opt.niter = 1000
opt.verbose = 0
opt.hessianScaledBeta = False
opt.gradTol = compute_grad_tol(tencer)
fixed_vars = [] 

In [ ]:
# Initialization: remove all compressed springs
t_init1 = time.time()
tencer = remove_compressed_springs(tencer,fixed_vars,opt)
t_init2 = time.time()
print("total time: ", t_init2 - t_init1)

In [ ]:
# Rigid registration of the target for visualization purposes
aligned_rod = target_registration(tencer,[rod])[0]

# Visualization
viewer_init = HybridViewer([tencer],wireframe=True)
v_init = Viewer(aligned_rod, superView=viewer_init)
viewer_init.show()

In [ ]:
# Save state for potential future use
# save_state(tencer,"data/trefoil_hypotrochoid_init.pkl")

# Greedy decimation: remove springs with lowest force

In [ ]:
# Load state if needed
# tencer = load_state("data/trefoil_hypotrochoid_init.pkl")

In [ ]:
# Rigid registration: align target
aligned_rods = target_registration(tencer,[aligned_rod])

In [ ]:
# Optimizer options
opt = NewtonOptimizerOptions()
opt.useNegativeCurvatureDirection = True
opt.niter = 1000
opt.gradTol = compute_grad_tol(tencer)
opt.verbose = 0
opt.hessianScaledBeta = False
fixed_vars = [] 

In [ ]:
# Greedy decimation

t_greedy1 = time.time()

# Set disance threshold
distance_threshold = 5e-6

tencer,aligned_rods = greedy_decimation_step(tencer,aligned_rods, target_registration, opt, fixed_vars,distance_threshold=distance_threshold)

t_greedy_2 = time.time()

In [ ]:
distance = distance_to_target(tencer,aligned_rods)
print("Normalized distance to the target: ", distance)
print("Number of springs: ", tencer.numRestVars())
print("Greedy decimation running time: ", t_greedy_2 - t_greedy1)

In [ ]:
# Visualization
viewer_greedy = HybridViewer([tencer],wireframe=True)
v_greedy = Viewer(aligned_rod, superView=viewer_greedy)
viewer_greedy.show()

In [ ]:
# Save state for potential future use
# save_state(tencer, "data/trefoil_hypotrochoid_greedy.pkl")

# Define symmetries (for optimization)

In [ ]:
# Load state if needed
# tencer = load_state("data/trefoil_hypotrochoid_greedy.pkl")

In [ ]:
# Find symmetric springs for a hypotrochoic trefoil with a 3-fold symmetry

def add_symmetric_springs(attachment_vertices, springs, num_vertices):
    
    symmetry_type = 3
    k = num_vertices / symmetry_type
    
    num_springs = len(attachment_vertices)
    spring_used = [False] * num_springs
    new_attachment_vertices = [] 
    new_springs = [] 
    
    for i in range(num_springs):
        if not spring_used[i]:
            
            # Get spring anchor points
            vA = attachment_vertices[i].vertexA
            vB = attachment_vertices[i].vertexB
            
            
            # Add this spring to the list
            new_attachment_vertices.append([0,vA,0,vB])
            stiffness = springs[i].stiffness 
            rest_length = springs[i].get_rest_length()
            new_springs.append(Spring(np.zeros(3),np.ones(3),stiffness,rest_length))
            
            # Get symmetric springs
            for m in range(symmetry_type-1):
                vS0 = int((vA + (m+1)*k)) % num_vertices
                vS1 = int((vB + (m+1)*k)) % num_vertices  
            
                # Add symmetric spring to the list
                new_attachment_vertices.append([0,vS0,0,vS1])
                new_springs.append(Spring(np.zeros(3),np.ones(3),stiffness,rest_length))

                # If the symmetric spring is in the list, mark it as used
                for j in range(i+1, num_springs):
                    vA_j = attachment_vertices[j].vertexA
                    vB_j = attachment_vertices[j].vertexB
                    if vA_j == vS0 and vB_j == vS1:
                        spring_used[j] = True
                    if vA_j == vS1 and vB_j == vS0:
                        spring_used[j] = True
                        
    symmetries = np.arange(len(new_springs)).reshape(-1,symmetry_type)
    a_v = []
    for x in new_attachment_vertices:
        a_v.append(SpringAttachmentVertices(x[0],x[1],x[2],x[3]))
        
    return new_springs,a_v,symmetries

In [ ]:
# Add symmetric springs
new_s,new_a,symmetries=add_symmetric_springs(tencer.getAttachmentVertices(),tencer.getSprings(), rod.numVertices())
tencer = Tencer([],tencer.getClosedRods(),new_s,new_a,target_rods)

In [ ]:
# Compute equilibrium
opt = NewtonOptimizerOptions()
opt.useNegativeCurvatureDirection = True
opt.niter = 100
opt.gradTol = compute_grad_tol(tencer)
opt.verbose = 1
fixed_vars = [rod.thetaOffset()]
c = computeEquilibrium(tencer,fixedVars=fixed_vars, opts=opt, hessianShift = 1e-8)

In [ ]:
# Check for compressed springs
v = tencer.getRestVars()
s = np.array(tencer.getSprings())
tensionned_springs = []
compressed_springs = []
for i,spring in enumerate(s):
    spring_length = np.linalg.norm(spring.get_coords()[:3] - spring.get_coords()[3:])
    spring_rest_length = spring.get_rest_length()
    if spring_length < spring_rest_length:
        compressed_springs.append(i)
    else: 
        tensionned_springs.append(i)
compressed_springs

In [ ]:
print("Number of springs: ", len(tencer.getSprings()))

In [ ]:
# Save state for potential future use
# save_state(tencer, "data/trefoil_hypotrochoid_symmetries.pkl")

# Replace cables with zero-rest length springs

In [ ]:
# Load state if needed
# tencer = load_state("data/trefoil_hypotrochoid_symmetries.pkl")

In [ ]:
# Set equilibrium options
opt = NewtonOptimizerOptions()
opt.useNegativeCurvatureDirection = True
opt.niter = 100
opt.gradTol = compute_grad_tol(tencer)
opt.verbose = 0
fixed_vars = [rod.thetaOffset()]

In [ ]:
# Get current rest lengths
rest_lengths = []
for spring in tencer.getSprings():
    rest_lengths.append(spring.get_rest_length())

In [ ]:
# Try reducing rest lengths while keeping the equilibrium stable
t_replace_springs1 = time.time()
for x in np.linspace(0,1,100):
    new_springs = replace_springs_with_rest_length(tencer.getSprings(),np.array(rest_lengths)*x)
    tencer1 = Tencer([],tencer.getClosedRods(),new_springs,tencer.getAttachmentVertices(),tencer.getTargetRods())
    lambdas, modes = compute_vibrational_modes.compute_vibrational_modes(tencer1, fixedVars=fixed_vars, mtype=compute_vibrational_modes.MassMatrixType.FULL, n=16, sigma=-1e-6)
    if lambdas[0]>-1e-10:
        x_opti = x
        print("Found minimal possible rest length: ",x)
        break 
t_replace_springs2 = time.time()
print("Running time: ", t_replace_springs2 - t_replace_springs1)

In [ ]:
# Create a new tencer with the new rest lengths
new_springs = replace_springs_with_rest_length(tencer.getSprings(),np.array(rest_lengths)*x_opti)
tencer = Tencer([],tencer.getClosedRods(),new_springs,tencer.getAttachmentVertices(),tencer.getTargetRods())

In [ ]:
# View tencer
viewer_rep = HybridViewer([tencer],wireframe=True)
v_rep = Viewer(aligned_rod, superView=viewer_rep)
viewer_rep.show()

In [ ]:
# Save state for potential future use
# save_state(tencer,"data/trefoil_hypotrochoid_replace_springs.pkl")

# Spring sparsification

In [ ]:
# Load state if needed
# tencer = load_state("data/trefoil_hypotrochoid_replace_springs.pkl")

In [ ]:
# Rigid registration
aligned_rods = target_registration(tencer,aligned_rods)

In [ ]:
# Equilibrium options used in the inner loop
opt_opts = NewtonOptimizerOptions()
opt_opts.useNegativeCurvatureDirection = True
opt_opts.niter = 1000
opt_opts.gradTol = 1e-5
opt_opts.verbose = 1
opt_opts.hessianScaledBeta = False
fixed_vars = [rod.thetaOffset()]

# This allows to provide weights to better approximate some areas of the target rod
# In the paper we only used uniform weights
radii_array = [np.ones(rod.numVertices())]

# Scipy optimization algorithm
algorithm = 'L-BFGS-B'
design_optimization_options = {'disp': True, 'maxiter':500}

# Callback function: remove rigid motion by realigning target rods with current rods
def realign(intermediate_result):
    global aligned_rods
    try: # seems to depend on scipy version
        tencer_opt.newPt(intermediate_result.x)
    except:
        tencer_opt.newPt(intermediate_result)
    aligned_rods = target_registration(tencer,aligned_rods)
    tencer_opt.update_target_rods([],aligned_rods,radii_array)

In [ ]:
# Sparsification weights - start with outer cables
# Start with 0: optimize shape before sparsifying
for weight in [0,1,10,1e2,1e3,1e4,1e5]:

    sparsification_weights = [weight]*tencer.numRestVars()

    # Define symmetry change of variable matrix
    ns = len(tencer.getSprings())
    symmetry_mat = TripletMatrix(ns,int(ns/3))
    for i in range(ns):
        symmetry_mat.addNZ(i,int(i/3),1)
    symmetry_mat = SuiteSparseMatrix(symmetry_mat)

    # Define optimization object
    tencer_opt = TencerOptimizationSymmetries(symmetry_groups = symmetry_mat,
                                          tencer=tencer,
                                          Newton_optimizer_options=opt_opts,
                                          fixed_vars=fixed_vars,
                                          open_target_rod=[],
                                          closed_target_rod=[aligned_rod],
                                          radii=radii_array,
                                          sparsification_weights = sparsification_weights,
                                          hessian_shift=1e-6)

    bound_constraints = Bounds(lb=np.zeros(len(tencer_opt.params())), keep_feasible=[True] * len(tencer_opt.params()))

    # Optimize
    t_sparsification1 = time.time()
    res = minimize(fun=tencer_opt.J,
                   x0=tencer_opt.params(),
                   jac=tencer_opt.gradp_J,
                   method=algorithm,
                   callback=realign,
                   bounds=bound_constraints,
                   options=design_optimization_options)
    t_sparsification2 = time.time()

    # Remove springs with zero stiffness
    tencer = remove_zero_springs(tencer)

    print("Running time: ", t_sparsification2 - t_sparsification1)
    print("Remaining springs: ", tencer.numRestVars())

In [ ]:
# Visualize results
print("Number of remaining springs:",len(tencer.getSprings()))
viewer_sp = HybridViewer([tencer],wireframe=True)
v_sp = Viewer(aligned_rods[0], superView=viewer_sp)
viewer_sp.show()

In [ ]:
# Save state for potential future use
# save_state(tencer,"data/trefoil_hypotrochoid_sparse_scipy.pkl")

# Optimize with sliding nodes

In [ ]:
# Load state if needed
# tencer = load_state("data/trefoil_hypotrochoid_sparse_scipy.pkl")

In [ ]:
# Rigid registration
aligned_rods = target_registration(tencer,aligned_rods)

In [ ]:
# Equilibrium options used in the inner loop
opt_opts = NewtonOptimizerOptions()
opt_opts.useNegativeCurvatureDirection = True
opt_opts.niter = 1000
opt_opts.gradTol = 1e-5
opt_opts.verbose = 0
opt_opts.hessianScaledBeta = False
fixed_vars = [rod.thetaOffset()]

# This allows to provide weights to better approximate some areas of the target rod
# In the paper we only used uniform weights
radii_array = [np.ones(rod.numVertices())]

# Here we are optimizing over the springs' positions: the rest variables type must be set to StiffnessAndSpringAnchors
tencer.setRestVarsType(RestVarsType.StiffnessAndSpringAnchors)

# Scipy optimization algorithm
algorithm = 'L-BFGS-B'
design_optimization_options = {'disp': True, 'maxiter':500, 'ftol': 1e-10}

# Callback function: remove rigid motion by realigning target rods with current rods
def realign(intermediate_result):
    global aligned_rods
    try: # seems to depend on scipy version
        tencer_opt.newPt(intermediate_result.x)
    except:
        tencer_opt.newPt(intermediate_result)
    aligned_rods = target_registration(tencer,aligned_rods)
    tencer_opt.update_target_rods([],aligned_rods,radii_array)

In [ ]:
# Define symmetries for spring stiffnesses and cable positions
L = rod.restLength()/3

# Define symmetry change of variable matrix
ns = len(tencer.getSprings())
symmetry_mat = TripletMatrix(3*ns,ns)
for i in range(ns):
    symmetry_mat.addNZ(i,int(i/3),1)
for i in range(ns,3*ns):
    symmetry_mat.addNZ(i,int(ns/3) + int((i-ns)/6) * 2 + (i%2),1)
symmetry_mat = SuiteSparseMatrix(symmetry_mat)

# Length offsets
length_offsets = np.zeros(3*ns)
l = np.zeros(2*ns)
rv = tencer.getRestVars()[ns:] - L
l[rv > -1e-6] += L
rv -= L
l[rv > -1e-6] += L
length_offsets[ns:] = l

In [ ]:
# Define optimization object
tencer_opt = TencerOptimizationSymmetries(symmetry_groups = symmetry_mat,
                                        tencer=tencer,
                                        Newton_optimizer_options=opt_opts,
                                        fixed_vars=fixed_vars,
                                        open_target_rod=[],
                                        closed_target_rod=[aligned_rod],
                                        radii=radii_array,
                                        length_offset = length_offsets,
                                        hessian_shift=1e-6)

bound_constraints = Bounds(lb=tencer_opt.getVarsLowerBounds(), ub = tencer_opt.getVarsUpperBounds(), keep_feasible=[True] * len(tencer_opt.params()))

# Optimize
t_opti1 = time.time()
res = minimize(fun=tencer_opt.J,
               x0=tencer_opt.params(),
               jac=tencer_opt.gradp_J,
               method=algorithm,
               callback=realign,
               bounds=bound_constraints,
               options=design_optimization_options)
t_opti2 = time.time()
print("Running time: ", t_opti2 - t_opti1)

In [ ]:
viewer_sn = HybridViewer([tencer],wireframe=True)
v_sn = Viewer(aligned_rod, superView=viewer_sn)
viewer_sn.show()

In [ ]:
# Save result to obj
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)
export_knot_to_obj(f"{output_dir}/trefoil_hypotrochoid_final_scipy", tencer)